In [3]:
import requests
import json
import en_core_web_md
from bs4 import BeautifulSoup
import re

In [5]:
nlp = en_core_web_md.load()

In [6]:
with open('./query_res.json') as f:
    dbpedia_data = json.load(f)

In [7]:
def get_page_content(title):
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': title,
            'prop': 'extracts',
            'exsectionformat': 'raw',
        }
    ).json()

    pages = [x['extract'] for x in response['query']['pages'].values()]
    html_text = '\n'.join(pages)
    soup = BeautifulSoup(html_text, 'html.parser')
    p = [x.text for x in soup.findAll('p')]
    q = [x.text for x in soup.findAll('blockquote')]
    content = '\n'.join(p + q)
    return content

In [281]:
# band_page_content = get_page_content('Katatonia')
band_page_content = get_page_content('Black_Sabbath')
# print(band_page_content)

In [282]:
# todo: try Sentencizer
# sents = [nlp(line) for line in re.split('(?<=\w\.{1}) ', band_page_content)]
sents = [line for line in re.split('(?<=\w\.{1}) ', band_page_content)]

In [311]:
# ['EP'] list of supported record types

def get_ents(doc):
    return [(e.text, e.label_) for e in doc.ents]


def get_ent(ents, t):
    r = [x for x in ents if x[0] == t]
    return r[0][1] if r else None

def get_parent_verb_for_date(token):
    if token.head and token.head.pos_ == 'VERB':
        return token.head.text
    return get_parent_verb_for_date(token.head)

def check_head(token, text):
    if token.head.text == token.text:
        return token.head.text == text
    return get_parent_verb_for_date(token.head)

def get_albums(sent):
    titles = re.findall('(([A-Z][a-z]+[\s\w\.\.\.\']+[A-Z][a-z]+)(?:\s+\(\d+\))?)', sent)
    if titles:
        yy = [x for x, _ in titles]
        doc = nlp(sent)
        org = [x for x in doc if x.ent_type_ == 'ORG']
#         print('&&&&', org)
        rrr = [x for x, _ in titles if any(org) in titles]
#         print('*&****', rrr)
            
    return [x for x, _ in titles]
    
def has_num(sent):
    return re.findall('\d{4}', sent)

def is_org(token):
    is_curr_org = token.ent_type_ == 'ORG'
    if not is_curr_org:
        return False
    is_begin_ent = token.ent_iob == 3
    is_inside_ent = token.ent_iob == 1
    if is_begin_ent:
        return [is_org(x) for x in token.children]
    else:
        return is_org(token.head)
    

def parse_sent(doc):
    for i in range(0, len(doc)):
        token = doc[i]
        if token.is_title == 'PROPN' or token.ent_type_ == 'ORG':


def get_al_o(doc, als):
    res = []
    for token in doc:
        if token.text in als and token.ent_type_ == 'ORG':
            res.append(token.text)
    return res

def get_album_det(string):
    res = re.findall('([A-Za-z\'\s]+)\((\d+)', string)
    return res[0] if res else None
                
    
# TODO: take all regex parsed 'titles', check doc for a sent if it contains title, check all words in title for: a) ORG, PERSON, PNOUN

In [313]:
all_albums = []
ss = sents[6:30]
for i in range(len(ss)):
    sent = ss[i]
    has_n = has_num(sent)
    if titles and has_n:
        albums = get_albums(sent)
        
        for album in albums:
            det = get_album_det(album)        
            if det:
                title, year = det
                all_albums.append({'title': title.strip(), 'year': year})
#         print(i, sent)
#         t = parse_sent(nlp(sent))
#         print('--', t)
#         print('====')
    
#     print('ht', ht)
#     parse_sent(nlp(sent))
s = nlp("In 1991, Iommi and Butler rejoined Dio and drummer Vinny Appice to record Dehumanizer (1992)")
for t in s:
    print(t.text, t.pos_, t.ent_type_)
print('!!!', all_albums)

In ADP 
1991 NUM DATE
, PUNCT 
Iommi PROPN PERSON
and CCONJ 
Butler PROPN PERSON
rejoined VERB 
Dio PROPN PERSON
and CCONJ 
drummer NOUN 
Vinny PROPN PERSON
Appice PROPN PERSON
to PART 
record VERB 
Dehumanizer PROPN PERSON
( PUNCT 
1992 NUM DATE
) PUNCT 
!!! [{'title': 'Iommi and Butler rejoined Dio and drummer Vinny Appice to record Dehumanizer', 'year': '1992'}]
